In [ ]:
! python -V

**Installations**

In [ ]:
! pip install kgforge

In [ ]:
! pip install allensdk

In [ ]:
! pip install git+https://github.com/BlueBrain/TMD

**Imports**

In [ ]:
from kgforge.core import KnowledgeGraphForge, Resource
from kgforge.specializations.resources import Dataset
from kgforge.specializations.mappings import DictionaryMapping
from kgforge.specializations.mappers import DictionaryMapper

In [1]:
from allensdk.core.cell_types_cache import CellTypesCache
from allensdk.api.queries.cell_types_api import CellTypesApi

In [ ]:
import tmd

**Setup**

In [ ]:
TOKEN = "<token>"

In [ ]:
forge = KnowledgeGraphForge.from_config("bbp-prod-forge.yml", bucket="bbp/<project>", token=TOKEN)

---

## User Journey A - Data Integration

1. Retrieve human neuron morphologies from the Allen Cell Types Database
2. Integrate these neuron morphologies into the Blue Brain Knowledge Graph
    - Retrieve the complete metadata of the selection of neuron morphologies
    - Integrate the Subject entities
    - Integrate the Patched Cell entities
    - Integrate the Neuron Morphology entities

### 1 - Retrieve human neuron morphologies from the Allen Cell Types Database

In [9]:
LIMIT = 2

In [10]:
cd ~/notebooks

/Users/fonta/notebooks


In [11]:
ALLEN_DIR = "allen_cell_types_db"

In [12]:
ctc = CellTypesCache(manifest_file=f"{ALLEN_DIR}/manifest.json")

In [13]:
human_cells = ctc.get_cells(species=[CellTypesApi.HUMAN], require_reconstruction=True)

In [14]:
human_cell_ids = [x["id"] for x in human_cells][:LIMIT]

In [15]:
human_cell_reconstructions = [ctc.get_reconstruction(x) for x in human_cell_ids]

### 2 - Integrate these neuron morphologies into the Blue Brain Knowledge Graph

Data integrated into the Blue Brain Knowledge Graph follow the Blue Brain Knowledge Graph Schema.

In the current case, this means that the data from the Allen Cell Types Database should be represented this way:

In [ ]:
# FIXME Image is inaccurate.

![Provenance](https://docs.google.com/uc?id=1Hoz3wK3vNkLxdhKNZXK53NOE6qKuBh7o)

In [ ]:
DATA_SOURCE = "allen-cell-types-database"

In [ ]:
forge.modeling.mappings(DATA_SOURCE)

In [ ]:
MAPPER = DictionaryMapper

**Retrieve the complete metadata of the selection of neuron morphologies**

In [17]:
import json

In [18]:
with open(f"{ALLEN_DIR}/cells.json") as f:
    allen_cell_types_metadata = json.load(f)

In [19]:
human_cell_metadata = [x for x in allen_cell_types_metadata if x["specimen__id"] in human_cell_ids]

In [20]:
human_cell_metadata[0]

{'cell_reporter_status': None,
 'csl__normalized_depth': 0.29095064121937,
 'csl__x': 79.0,
 'csl__y': 273.0,
 'csl__z': 91.0,
 'donor__age': '39 yrs',
 'donor__disease_state': 'epilepsy',
 'donor__id': 518641172,
 'donor__name': 'H16.03.001',
 'donor__race': 'White or Caucasian',
 'donor__sex': 'Male',
 'donor__species': 'Homo Sapiens',
 'donor__years_of_seizure_history': '19',
 'ef__adaptation': 0.179194027185622,
 'ef__avg_firing_rate': 4.06658352762565,
 'ef__avg_isi': 245.906666666667,
 'ef__f_i_curve_slope': 0.0571428571428571,
 'ef__fast_trough_v_long_square': -47.0,
 'ef__peak_t_ramp': 9.88393333333333,
 'ef__ri': 146.874636411667,
 'ef__tau': 50.0490308584905,
 'ef__threshold_i_long_square': 130.0,
 'ef__upstroke_downstroke_ratio_long_square': 4.57486453842852,
 'ef__vrest': -72.5476608276367,
 'ephys_inst_thresh_thumb_path': '/api/v2/well_known_file_download/522910216',
 'ephys_thumb_path': '/api/v2/well_known_file_download/522910214',
 'erwkf__id': 618213231,
 'line_name': '

**Integrate the Subject entities**

In [ ]:
subject_mapping = forge.modeling.mapping("Subject", DATA_SOURCE)

In [ ]:
subjects = forge.transforming.map(human_cell_metadata, MAPPER, subject_mapping)

In [ ]:
forge.modeling.check(subjects)

In [ ]:
forge.storing.register(subjects)

**Integrate the Patched Cell entities**

In [ ]:
patchedcell_mapping = forge.modeling.mapping("PatchedCell", DATA_SOURCE)

In [ ]:
patchedcells = forge.transforming.map(human_cell_metadata, MAPPER, patchedcell_mapping)

In [ ]:
forge.modeling.valdiate(patchedcells)

In [ ]:
forge.storing.register(patchedcells)

**Integrate the Neuron Morphology entities**

In [ ]:
neuronmorphology_mapping = forge.modeling.mapping("NeuronMorphology", DATA_SOURCE)

In [ ]:
neuronmorphologies = forge.transforming.map(human_cell_metadata, MAPPER, neuronmorphology_mapping)

In [ ]:
forge.modeling.check(neuronmorphologies)

In [ ]:
forge.storing.register(neuronmorphologies)

---

## User Journey B - Data Exploration

1. Discover which neuron morphologies are in the Blue Brain Knowledge Graph
2. Regroup as a dataset a selection of these neuron morphologies
    - Select neuron morphologies in the cortical layer V and with intact apical dendrites
    - Register the selected neuron morphologies as a dataset
    - Give this first revision of the dataset a human-friendly name

In [ ]:
p = forge.modeling.paths("NeuronMorphology")

### 1 - Discover which neuron morphologies are in the Blue Brain Knowledge Graph

In [ ]:
results = forge.querying.search(p.type == "NeuronMorphology")

In [ ]:
len(results)

### 2 - Regroup as a dataset a selection of these neuron morphologies

**Select neuron morphologies in the cortical layer V and with intact apical dendrites**

In [ ]:
neuronmorphologies = forge.querying.search(p.type == "NeuronMorphology", p.brainLocation.layer == "5",
                                           p.apicalDendrite == "intact", resolving="fuzzy", lookup="children")

In [ ]:
forge.transforming.as_dataframe(neuronmorphologies)

**Register the selected neuron morphologies as a dataset**

In [ ]:
AGENT_ID = "<contributor ID>"

In [ ]:
# FIXME According to Neuroshapes on 09.08.2019, 'type', 'subject', 'brainLocation' are also required.
# FIXME Property 'hasPart' is not part of the Dataset shape at the moment (09.08.2019).
dataset = Dataset(forge,
                  name="All layer 5 morphologies with intact apical dendrites",
                  description="Neuron morphologies to be used for Topological Morphology Descriptor analysis")

In [ ]:
dataset.add_contribution(AGENT_ID)

In [ ]:
dataset.add_parts(neuronmorphologies)

In [ ]:
forge.storing.register(dataset)

**Give the first revision of the dataset a human-friendly label**

In [ ]:
TAG = "v2019-08-20"

In [ ]:
forges.storing.tag(dataset, TAG)

---

## User Journey C - Data Analytics

1. Retrieve a specific dataset from the Blue Brain Knowledge Graph
    - Retrieve the dataset entity
    - Download the reconstruction files of the neuron morphologies of the dataset
2. Perform a topological analysis of the neuron morphologies from the dataset
    - Visualize the persistence diagram
    - Visualize the persistence barcode
    - Visualize and save the persistence image
3. Register the analysis result with its provenance into the Blue Brain Knwoledge Graph

In [ ]:
DOWNLOAD_FOLDER = f"./reconstructions_{TAG}/"

In [ ]:
ANALYSIS_FOLDER = "./analysis/"

In [ ]:
ANALYSIS_FILENAME = "persistence_image.png"

### 1 - Retrieve a specific dataset from the Blue Brain Knowledge Graph

**Retrieve the dataset entity**

In [ ]:
dataset = forge.querying.retrieve(id="<dataset ID>", version=TAG)

**Download the reconstruction files of the neuron morphologies of the dataset**

In [ ]:
dataset.download("parts", DOWNLOAD_FOLDER)

### 2 - Perform a topological analysis of the neuron morphologies from the dataset

In [ ]:
from pathlib import Path

In [ ]:
Path(DOWNLOAD_FOLDER).mkdir(exist_ok=True)

In [ ]:
pop = tmd.io.load_population(DOWNLOAD_FOLDER)

In [ ]:
phs = [tmd.methods.get_persistence_diagram(x.apical[0]) for x in pop.neurons]

In [ ]:
phs_flattened = tmd.analysis.collapse(phs)

**Visualize the persistence diagram**

In [ ]:
from tmd.view import plot

In [ ]:
plot.diagram(phs_flattened)

**Visualize the persistence barcode**

In [ ]:
plot.barcode(phs_flattened)

**Visualize and save the persistence image**

In [ ]:
plot.persistence_image(phs_flattened, output_path=ANALYSIS_FOLDER, output_name=ANALYSIS_FILENAME)

### 3 - Register the analysis result with its provenance into the Blue Brain Knwoledge Graph

In [ ]:
# FIXME According to Neuroshapes on 09.08.2019, 'used' (2 times) ans 'generated' are also required.
analysis = Dataset(forge, type="Analysis", name="Persistence image")

In [ ]:
analysis.add_contribution(AGENT_ID)

In [ ]:
analysis.add_derivation(dataset)

In [ ]:
analysis.add_files(f"./{ANALYSIS_FOLDER}/{ANALYSIS_FILENAME}")

In [ ]:
forge.storing.register(analysis)